In [ ]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def cbb_calendar(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates={}".format(season)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
    datenum = list(map(lambda x: x[:10].replace("-",""),txt))
    date = list(map(lambda x: x[:10],txt))
    
    year = list(map(lambda x: x[:4],txt))
    month = list(map(lambda x: x[5:7],txt))
    day = list(map(lambda x: x[8:10],txt))
    
    data = {"season": season,
            "datetime" : txt,
            "date" : date,
            "year": year,
            "month": month,
            "day": day,
            "dateURL": datenum
            
           }
    df = pd.DataFrame(data)
    df['url']="http://site.api.espn.com/apis/site/v2/sports/football/college-football/scoreboard?dates="
    df['url']= df['url'] + df['dateURL']
    return df

cbb_calendar(2018)

In [ ]:
pd.set_option('display.max_colwidth',None)
schedule = {}
for x in range(2001,2022):
    full_schedule = cbb_calendar(x)
    full_schedule.to_json(orient='records')
    schedule[x]=full_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('cbb_schedule.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)    

In [ ]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def cbb_schedule(year):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates={}".format(year)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
#     print(len(txt))
    txt = list(map(lambda x: x[:10].replace("-",""),txt))
    
    ev = pd.DataFrame()
    i=0
    for date in txt:
        print(f"Working on {year}: {i+1} of {len(txt)}, date: {txt[i]}")
        url = "http://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?groups=50&dates={}".format(date)
        resp = download(url=url)
        events_txt = json.loads(resp)
        events = events_txt['events']
        for event in events:
            bad_keys = ['linescores', 'statistics', 'leaders',  'records']
            for k in bad_keys:
                if k in event['competitions'][0]['competitors'][0].keys():
                    del event['competitions'][0]['competitors'][0][k]
                if k in event['competitions'][0]['competitors'][1].keys():
                    del event['competitions'][0]['competitors'][1][k]
            if 'links' in event['competitions'][0]['competitors'][0]['team'].keys():
                del event['competitions'][0]['competitors'][0]['team']['links']
            if 'links' in event['competitions'][0]['competitors'][1]['team'].keys():
                del event['competitions'][0]['competitors'][1]['team']['links']    
            if event['competitions'][0]['competitors'][0]['homeAway']=='home':
                event['competitions'][0]['home'] = event['competitions'][0]['competitors'][0]['team']    
            else: 
                event['competitions'][0]['away'] = event['competitions'][0]['competitors'][0]['team']
            if event['competitions'][0]['competitors'][1]['homeAway']=='away':
                event['competitions'][0]['away'] = event['competitions'][0]['competitors'][1]['team']
            else: 
                event['competitions'][0]['home'] = event['competitions'][0]['competitors'][1]['team']

            del_keys = ['competitors', 'broadcasts','geoBroadcasts', 'headlines']
            for k in del_keys:
                if k in event['competitions'][0].keys():
                    del event['competitions'][0][k]

            ev = ev.append(pd.json_normalize(event['competitions'][0]))
        i+=1
    ev['season']=year
    return ev

schedule_table = pd.DataFrame()
schedule = {}
for x in range(2002,2022):
    year_schedule = cbb_schedule(x)
    year_schedule.to_csv(f"cbb_games_info_{x}.csv")
    schedule_table = schedule_table.append(year_schedule)
    year_schedule.to_json(orient='records')
    schedule[x]=year_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('cbb_schedule.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)

schedule_table.to_csv('cbb_games_info_2002_2021.csv')
    
# event['competitions'][0]